# Wordcount example

This notebook shows the classic wordcount example in which we want to calculate how many time the same word appears within a text.
The example also show how to use basic PySpark tools like DataFrame and SQL.

In [ ]:
# To find out where the pyspark
import findspark
findspark.init()

In [ ]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext("local", "Wordcount")

With the step below we are going to read an input (local) file that will be our data source. textFile() and wholeTextFiles() methods to read into RDD that are the low level data access of Spark (there exist other method to read directly in Dataframe).

Each line of the text file is a *row*. We can apply a series of chained operation:
1. flatMap produces a new dataset <word> from the splitting
2. map produces a new dataset in the form <word, couunt>
3. reduceByKey coordinates the aggregation by summing rows with the same key

In [ ]:
# Calculating words count
text_file = sc.textFile("doveconviene_info.txt", numPartitions=10)
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

We now print some basec statistics about words and their occurrences

In [ ]:
# Printing each word with its respective count
output = counts.collect()
for (word, occurs) in output:
    print("%s: %i" % (word, occurs))

In [ ]:

counts.countByKey()

## Dataframe in PySpark

PySpark is Python on Spark, programming language and syntax are the ones that we already know.

*Dataframe* is the approach to analysis via high-level data structures that work in a distributed way. DataFrames is a data structure already known in Python/R that allow you to have a tabular/columnar form of the data. Operations are simple to execute and runs in parallel.

Convert to a friendly structure: dataframe
https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [ ]:
from pyspark.sql import Row # import the pyspark sql Row class

rows = counts.map(lambda p: Row(word=p[0], occurs=int(p[1]))) # tuples -> Rows
#rows.toDF().createOrReplaceTempView("word_count")

In [ ]:
df = rows.toDF()

In [ ]:
df.printSchema()

In [ ]:
df.head(5)

In [ ]:
df.show(10,truncate= True) 
# https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show

In [ ]:
df.count()

In [ ]:
len(df.columns), df.columns


In [ ]:
df.describe().show()

In [ ]:
df.describe('occurs').show()

In [ ]:
df.select('occurs','word').show(5)


In [ ]:
df.select('word').show(5)

In [ ]:
df.select('word').distinct().count(), df.select('word').count(),df.select('word').dropDuplicates().count()

In [ ]:
df.orderBy(df['occurs'].desc()).show(500)

## Moving to SQL

We can also use SQL syntax to analyze Dataframe.We first need to define a virtual SQL table from the dataframe. 


In [ ]:
 word_count = df.createOrReplaceTempView("word_count")

In [ ]:
spark.sql('select * from word_count order by occurs DESC ').show(5)


In [ ]:
spark.sql("""
select                
* 
from word_count
WHERE length(word) > 2
ORDER BY occurs  DESC
""").show(5)

In [ ]:
# Finally, save our result
df.write.mode('overwrite').option("header", "true").save('wordcount.csv',format='csv')

In [ ]:
# Stopping Spark Context
sc.stop()

## Consideration about Dataframe and SQL

Dataframes are interchangeable with SQL. We can register a DataFrame as a SQL table and query it via SQL syntax.

Benefits? High level access to the dataset. DataFrame as a SQL have similar performances. RDD (low level access) is fastest than Dataframe and SQL.
More info here: 
https://community.cloudera.com/t5/Community-Articles/Spark-RDDs-vs-DataFrames-vs-SparkSQL/ta-p/246547
https://stackoverflow.com/questions/45430816/writing-sql-vs-using-dataframe-apis-in-spark-sql


## What's next?

### Dataframe

Good operation overview.
https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

The basic operations on Dataframe are described in terms operation requirements.
https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

### SQL
SparkSql follows Hive style, so you can refer to Hive Syntax for documentation.
https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select

The supported and unsupported Hive features by SparkSql can be found in the official documentation.
https://spark.apache.org/docs/latest/sql-programming-guide.html#compatibility-with-apache-hive

### SparkExamples

Below a reference site for a good hoverview of basic Spark operations (cluster setup, RDD operations, Dataframe operations, SQL operations, Streaming, integration with other frameworks).
It is for Scala but it will be easy get feedback also for PySpark.
https://sparkbyexamples.com/![image.png](attachment:image.png)
